In [1]:
from easyDAG import do_eval, Step, InputVariable

In [2]:
import pandas as pd

In [4]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
a = InputVariable('a')
b = InputVariable('b')
c = InputVariable('c')
result = pd.merge(left, right, on='key')
cas_result = Step(pd.merge, a, b, on='key')
result_post = do_eval(cas_result, a=left, b=right)
assert (result == result_post).all().all()

In [3]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                    'key2': ['K0', 'K1', 'K0', 'K1'],
                    'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                     'key2': ['K0', 'K0', 'K0', 'K0'],
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']})
a = InputVariable('a')
b = InputVariable('b')
c = InputVariable('c')
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
cas_result = Step(pd.merge, a, b, how='left', on=c)
result_post = do_eval(cas_result, a=left, b=right, c=['key1', 'key2'])
assert (result.fillna(0) == result_post.fillna(0)).all().all()

In [5]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
X = [[ 1,  2,  3], [11, 12, 13]]
y = [0, 1]  # classes of each sample
X2 = [[4, 5, 6], [14, 15, 16]]

x1_v = InputVariable('X')
x2_v = InputVariable('X_new')
y_v = InputVariable('y')

make_rfc = Step(RandomForestClassifier, random_state=0, n_estimators=10)
fitted_clf = make_rfc.fit(X=x1_v, y=y_v)
predict = fitted_clf.predict(X=x2_v)
result = do_eval(predict, X=X, X_new=X2, y=y)

clf_2 = RandomForestClassifier(random_state=0, n_estimators=10)
assert np.all(result == clf_2.fit(X, y).predict(X2))

make_rfc = Step(RandomForestClassifier, random_state=0, n_estimators=10)
fitted_clf = make_rfc.fit(x1_v, y_v)
predict = fitted_clf.predict(x2_v)
result = do_eval(predict, X=X, X_new=X2, y=y)

clf_2 = RandomForestClassifier(random_state=0, n_estimators=10)
assert np.all(result == clf_2.fit(X, y).predict(X2))